In [292]:
import pandas as pd
import numpy as np
import warnings
from functools import partial

In [293]:
from __future__ import annotations
from typing import List

class Priogrid(object):

    def __init__(self, id: int):
        self.__validate_id(id)
        self.id = id
        self.row = self.id2row(id)
        self.col = self.id2col(id)
        self.lat = self.id2lat(id)
        self.lon = self.id2lon(id)

    def __repr__(self):
        return f'Priogrid({self.id})'

    def __str__(self):
        return f'Priogrid(id={self.id}) #=> row:{self.row}, col:{self.col}, lat:{self.lat}, lon:{self.lon}'

    @classmethod
    def from_lat_lon(cls, lat: float, lon: float) -> Priogrid:
        """
        A factory producing a Priogrid Object from a lat and lon
        :param lat: A WGS84 valid lat (i.e. -90..90)
        :param lon: A WGS84 valid lon (i.e. -180..180)
        :return: A Priogrid Object at that position
        """
        return cls(cls.latlon2id(lat, lon))


    @classmethod
    def from_row_col(cls, row: int, col: int) -> Priogrid:
        """
        A factory producing a Priogrid Object from a row and col
        :param row: A Priogrid valid row (i.e. 0..360)
        :param col: A Priogrid valid col (i.e. 0..720)
        :return: A Priogrid Object at the (row,col) position
        """
        return cls(cls.rowcol2id(row, col))

    def next_right(self) -> Priogrid:
        """
        A factory method returning a Priogrid Object situated to the RIGHT of the currentone
        :return: A Priogrid Object
        """
        if self.col < 720:
            return Priogrid.from_row_col(self.row, self.col+1)
        else:
            return None

    def next_left(self) -> Priogrid:
        if self.col > 1:
            return Priogrid.from_row_col(self.row, self.col-1)
        else:
            return None

    def next_down(self) -> Priogrid:
        if self.row > 1:
            return Priogrid.from_row_col(self.row-1, self.col)
        else:
            return None

    def next_up(self) -> Priogrid:
        if self.row < 360:
            return Priogrid.from_row_col(self.row+1, self.col)
        else:
            return None

    def rook_contiguity(self) -> List[List[Priogrid]]:
        """
        Factory object, producing the rook contiguity matrix (3x3 cross convolution kernel) of the current object
        Objects outside of edges return None.
        :return: A matrix (2-D List of lists) of Priogrid objects representing the rook contiguity matrix
        """
        up = [None, self.next_up(), None]
        center = [self.next_left(), self, self.next_right()]
        down = [None, self.next_down(), None]
        return [up, center, down]

    def queen_contiguity(self) -> List[List[Priogrid]]:
        """
        Factory object, producing the queen contiguity matrix (3x  convolution kernel) of the current object
        Objects outside of edges return None.
        :return: A matrix (2-D List of lists) of Priogrid objects representing the queen contiguity matrix
        """

        queen = self.rook_contiguity()
        if queen[0][1] is not None:
            queen[0][0] = queen[0][1].next_left()
            queen[0][2] = queen[0][1].next_right()
        if queen[2][1] is not None:
            queen[2][0] = queen[2][1].next_left()
            queen[2][2] = queen[2][1].next_right()
        return queen

    @classmethod
    def id2lat(cls, id):
        """
        Return a centroid latitude for a given id.
        :param id: id
        :return: a centroid (x.25) lat position for the cell identified by id
        """
        return cls.row2lat(cls.id2row(id))

    @classmethod
    def id2lon(cls, id):
        """
        Return a centroid longitude for a given id.
        :param id: id
        :return: a centroid (x.25) lon position for the cell identified by id
        """
        return cls.col2lon(cls.id2col(id))

    @classmethod
    def latlon2id(cls, lat, lon, hard=True):
        """
        Returns a Priogrid ID from a lat and lon set of floats
        :param lat: latitude
        :param lon: longitude
        :return: a pg id
        """
        cls.__validate_lat(lat)
        cls.__validate_lon(lon)
        row = cls.lat2row(lat)
        col = cls.lon2col(lon)
        id = cls.rowcol2id(row, col)
        return id

    @staticmethod
    def id2row(id):
        return int(id / 720)+1

    @staticmethod
    def id2col(id):
        return id % 720

    @staticmethod
    def rowcol2id(row, col):
        return (row-1)*720+col
    
    @staticmethod
    def __validate_id(id):
        if not(0<=id<=259200):
            raise ValueError("ID must be between 1 and 259200")
    
    @staticmethod
    def __validate_lat(lat):
        if not(-90<=lat<=90):
            raise ValueError("Latitude must be within the [-90;90] interval")
    
    @staticmethod
    def __validate_lon(lon):
        if not(-180<=lon<=180):
            raise ValueError("Latitude must be within the [-180;180] interval")

    @staticmethod
    def lat2row(lat):
        Priogrid.__validate_lat(lat)
        return int(abs(-90 - lat) / 0.5) + 1
    
    @staticmethod
    def lon2col(lon):
        Priogrid.__validate_lon(lon)
        return int(abs(-180 - lon) / 0.5) + 1
        
    @staticmethod
    def col2lon(col):
        return (-180+(col*0.5))-0.25

    @staticmethod
    def row2lat(row):
        return (-90+(row*0.5))-0.25

In [302]:
pg_object.queen_contiguity()[1][1].lat

12.75

In [80]:
#Priogrid(100000).lat
try:
    Priogrid(2043995)
except ValueError:
    None

In [317]:
class ViewsMonth(object):
    
    def __init__(self,id):
        self.__validate(id)
        self.id = int(id)
        self.month = self.id2month(id)
        self.year = self.id2year(id)

        
    def __repr__(self):
        return f'ViewsMonth({self.id})'

    def __str__(self):
        return f'ViewsMonth(id={self.id}) #=> year:{self.year}, month:{self.month}'
    
    @classmethod
    def id2month(cls, id):
        return (id-1)%12+1
        
    @classmethod
    def id2year(cls, id):
        return int((id-1)/12)+1980
    
    @staticmethod
    def __validate(id):
        if int(id)<=0:
            raise ValueError("Monthid cannot be negative")
    
    @staticmethod
    def __validate_year(year):
        if year<1980:
            raise ValueError("Year must be >=1980")
    
    def __validate_month(month):
        if not(1 <= month <= 12):
            raise ValueError("Month must be between 1 and 12")
    
    @classmethod
    def from_year_month(cls, year, month):
        """
        A factory returning a ViewsMonth object with the proper id.
        """
        cls.__validate_year(year)
        cls.__validate_month(month)
        return cls(int((year-1980)*12+month))

In [321]:
str(ViewsMonth(0))

ValueError: Monthid cannot be negative

In [21]:
str(ViewsMonth.from_year_month(1989,4))

'ViewsMonth(id=112) #=> year:1989, month:4'

In [83]:
ViewsMonth.from_year_month(1997,1)

ViewsMonth(205)

In [84]:
ViewsMonth.from_year_month(1997,12)

ViewsMonth(216)

In [274]:
@pd.api.extensions.register_dataframe_accessor("pg")
class PgAccessor:
    def __init__(self, pandas_obj):
        self._validate(pandas_obj)
        self._obj = pandas_obj
        self._obj.pg_id = self._obj.pg_id.astype('int')
    
    @staticmethod
    def _validate(obj):
        #print (obj.columns)
        if "pg_id" not in obj.columns:
            raise AttributeError("Must have a pg_id column!")
        mid = obj.pg_id.copy()
        if mid.dtypes != 'int':
            warnings.warn('pg_id is not an integer - will try to typecast in place!')
            mid = mid.astype('int')
        if mid.min() < 1:
            raise ValueError("Negative pg_id encountered!")
        if mid.max() > 259200:
            raise ValueError("pg_id out of bounds!")

    @property
    def is_unique(self):
        """
        returns True is ...
        """
        if pd.unique(self._obj.pg_id).size == self._obj.pg_id.size:
            return True
        return False
    
    @property
    def lat(self):
        return self._obj.apply(lambda x: Priogrid(x.pg_id).lat, axis=1)
    
    @property
    def lon(self):
        return self._obj.apply(lambda x: Priogrid(x.pg_id).lon, axis=1)
        
    @classmethod
    def from_latlon(cls, df, lat_col='lat', lon_col='lon'):
        z = df.copy()
        z['pg_id']=df.apply(lambda row: Priogrid.latlon2id(lat=row[lat_col],lon=row[lon_col]),axis=1)
        return z
    
    @staticmethod
    def __soft_validate_pg(row, lat_col, lon_col):
        try:
            id = Priogrid.latlon2id(lat=row[lat_col],lon=row[lon_col])
            ok = True
        except ValueError:
            ok = False
        return ok
    
    @classmethod
    def soft_validate_latlon(cls, df, lat_col='lat', lon_col='lon'):
        z = df.copy()
        soft_validator = partial(PgAccessor.__soft_validate_pg, lat_col=lat_col, lon_col=lon_col)
        z['valid_latlon'] = z.apply(soft_validator, axis=1)
        return z
        

<ipython-input-274-9319d6f63c1a>:2: UserWarning: registration of accessor <class '__main__.PgAccessor'> under name 'pg' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class PgAccessor:


In [267]:
@pd.api.extensions.register_dataframe_accessor("m")
class MAccessor():
    def __init__(self, pandas_obj):
        self._validate(pandas_obj)
        self._obj = pandas_obj
        self._obj.month_id = self._obj.month_id.astype('int')
    
    @staticmethod
    def _validate(obj):
        #print (obj.columns)
        if "month_id" not in obj.columns:
            raise AttributeError("Must have a month_id column!")
        mid = obj.month_id.copy()
        if mid.dtypes != 'int':
            warnings.warn('month_id is not an integer - will try to typecast in place!')
            mid = mid.astype('int')
        if mid.min() < 1:
            raise ValueError("Negative month_id encountered")

    @property
    def is_unique(self):
        if pd.unique(self._obj.month_id).size == self._obj.month_id.size:
            return True
        return False
    
    @property
    def year(self):
        return self._obj.apply(lambda x: ViewsMonth(x.id).year, axis=1)
    
    @property
    def month(self):
        return self._obj.apply(lambda x: ViewsMonth(x.id).month, axis=1)
    
    @classmethod
    def from_year_month(cls, df, year_col='year', month_col='month'):
        z = df.copy()
        z['month_id']=z.apply(lambda row: ViewsMonth.from_year_month(year=row[year_col],
                                                                      month=row[month_col]).id,
                               axis=1)
        return z
    
    @staticmethod
    def __soft_validate_month(row, year_col, month_col):
        try: 
            id = ViewsMonth.from_year_month(year=row[year_col],month=row[month_col]).id
            ok = True
        except ValueError:
            ok = False
        return ok
                
    @classmethod
    def soft_validate_year_month(cls, df, year_col='year', month_col='month'):
        z = df.copy()
        soft_validator = partial(MAccessor.__soft_validate_month, year_col=year_col, month_col=month_col)
        z['valid_year_month'] = z.apply(soft_validator, axis=1)
        return z

<ipython-input-267-285d135d3b5c>:2: UserWarning: registration of accessor <class '__main__.MAccessor'> under name 'm' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class MAccessor():


In [290]:
@pd.api.extensions.register_dataframe_accessor("pgm")
class PGMAccessor(PgAccessor, MAccessor):
    def __init__(self, pandas_obj):
        PgAccessor._validate(pandas_obj)
        MAccessor._validate(pandas_obj)
        self._obj = pandas_obj
        self._obj.month_id = self._obj.month_id.astype('int')
        self._obj.pg_id = self._obj.pg_id.astype('int')
    
    @property
    def is_unique(self):
        uniques = self._obj[['pg_id','month_id']].drop_duplicates().shape[0]
        totals = self._obj.shape[0]
        if  uniques == totals:
            return True
        return False
    
    @classmethod
    def from_year_month_latlon(cls, df, year_col='year', month_col='month', lat_col='lat', lon_col='lon'):
        z = df.copy()
        z = super().from_year_month(z,year_col=year_col, month_col=month_col)
        z = super().from_latlon(z,lat_col=lat_col,lon_col=lon_col)
        return z
    
    @classmethod
    def soft_validate_year_month_latlon(cls, df, year_col='year', month_col='month', lat_col='lat', lon_col='lon'):
        z = df.copy()
        z = super().soft_validate_year_month(z, year_col=year_col, month_col=month_col)
        z = super().soft_validate_latlon(z, lat_col=lat_col, lon_col=lon_col)
        z['valid_year_month_latlon'] = z.valid_year_month & z.valid_latlon
        return z


<ipython-input-290-df166fe6721b>:2: UserWarning: registration of accessor <class '__main__.PGMAccessor'> under name 'pgm' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class PGMAccessor(PgAccessor, MAccessor):


In [291]:
@pd.api.extensions.register_dataframe_accessor("c")
class PGMAccessor():
    def __init__(self, pandas_obj):
        self._validate(pandas_obj)
        self._obj = pandas_obj
        self._obj.month_id = self._obj.month_id.astype('int')
        
    def _validate(obj):
        pass


In [192]:
pgm1 = pd.DataFrame({"pg_id":[49141,39218,53959,53959],"month_id":[294,293,294,294],"data":[1,2,4,6]})
pgm2 = pd.DataFrame({"pg_id":[49141,39218,891,53959],"month_id":[294,293,294,294],"data":[1,2,4,6]})

In [286]:
pgm5 = pd.DataFrame({'year':[1991,1995,1981],'month':[11,12,1],'lat':[9.2,4.2,11.2],'lon':[200.5,22.5,29.5]})

In [289]:
pd.DataFrame.pgm.soft_validate_year_month_latlon(pgm5)

,year,month,lat,lon,valid_year_month,valid_latlon,valid_year_month_latlon
0,1991,11,9.2,200.5,True,False,False
1,1995,12,4.2,22.5,True,True,True
2,1981,1,11.2,29.5,True,True,True


In [195]:
m1 = pd.DataFrame({"month_id":[100,101,102]})
m2 = pd.DataFrame({"month_id":[100,100,102]})
m3 = pd.DataFrame({"month_id":['8','349']})
m4 = pd.DataFrame({"month_id":['8','9229','e']})

In [150]:
assert pgm2.pgm.is_unique == True
assert pgm1.pgm.is_unique == False

In [266]:
m5 = pd.DataFrame({'year':[1991,1995,1981],'month':[11,36,1]})
m6 = pd.DataFrame.m.soft_validate_year_month(m5)
m6

,year,month,valid_year_month
0,1991,11,True
1,1995,36,False
2,1981,1,True


In [313]:
str(ViewsMonth(0))

'ViewsMonth(id=0) #=> year:1980, month:12'

In [238]:
am3.month_id.dtype

dtype('O')

In [29]:
assert m1.m.is_unique == True
assert m2.m.is_unique == False

In [162]:
x = pd.DataFrame({"pg_id":[145294,145295], 'value':[100,300]})
y = pd.DataFrame({"pg_id":[100,101,103,103]})
k = pd.DataFrame({"lat":[2,32.4,32.4],"lon":[23,42,42]})

In [204]:
x = pd.DataFrame.pgm.from_year_month(pd.DataFrame.pgm.from_latlon(pgm5))

In [206]:
x.pgm.is_unique

True

In [273]:
pd.DataFrame.pg.soft_validate_latlon(k)

AttributeError: type object 'MAccessor' has no attribute '_PgAccessor__soft_validate_pg'

In [327]:
k = pd.DataFrame({"lat":[2,32.4,32.4],"lon":[23203,42,42]})
k = pd.DataFrame.pg.from_latlon(k)
k

,lat,lon,pg_id
0,2.0,23203,179247
1,32.4,42,176125
2,32.4,42,176125


In [275]:
k = pd.DataFrame({"lat":[2,32.4,52.4],"lon":[923.7,42.3,42.1],"value":[5,29,21]})

In [264]:
k

,lat,lon,value
0,2.0,923.7,5
1,32.4,42.3,29
2,52.4,42.1,21


In [277]:
pd.DataFrame.pg.soft_validate_latlon(k)

,lat,lon,value,valid_latlon
0,2.0,923.7,5,False
1,32.4,42.3,29,True
2,52.4,42.1,21,True


In [284]:
aaa = pd.DataFrame({'a':[False,True,False,True],'b':[False,False,True,True]})
aaa['c']=aaa.a & aaa.b
aaa

,a,b,c
0,False,False,False
1,True,False,False
2,False,True,False
3,True,True,True


In [276]:
pg_k = pd.DataFrame.pg.from_latlon(k)

ValueError: Latitude must be within the [-180;180] interval

In [266]:
pg_k

,lat,lon,value,pg_id
0,2.0,923.7,5,134688
1,32.4,42.3,29,176125
2,52.4,42.1,21,204925


In [267]:
pg_k.pg.is_unique#unique

True

In [253]:
df.pg.views_write()

False

In [ ]:
pd.DataFrame.pg.from_latlon(k)

In [230]:
assert x.pg.is_unique == True
assert y.pg.is_unique == False
assert pd.DataFrame.pg.from_latlon(k).pg_id.is_unique == False
assert pd.DataFrame.pg.from_latlon(k).pg.lat[1] == 32.25 #This should be the centroid of the PG!